In [135]:
import pandas as pd
import re
import numpy as np
import operator
import functools
from sklearn.feature_extraction.text import CountVectorizer

datadir = '/home/xavier/data/'

In [136]:
# clinvar_stub = pd.read_csv(datadir + 'clinvar_multiple-diseases-listed.csv',nrows=50)
# clinvar_stub.shape

In [137]:
# list(clinvar_stub)

### So this is about two things

1. Understanding 'Enumberate'
2. Matching using the methods in these two examples:

    + https://stackoverflow.com/questions/42683249/python-pandas-how-to-match-list-of-strings-with-a-dataframe-column
    
- - - 

## First, enumerate
    

In [138]:
firstdata = {'TAG': ['a,b,f','b','c,g,a','b,d,f,h','h','c,k'],
       'DESC': ['Apples,Bananas,Fruits','Bananas','Citron,Guava,Apple', \
                'Bananas,Dilithium,Fruits,Hottentot','Hottentot','Citron,Kiwi']}

# checklist = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r']
# checklist = ['a','b','c','d','e','f','g','h','i','j','k','l']
checklist = ['a','b','c','d','e','h','i','j','k','l']

In [139]:
first = pd.DataFrame(firstdata)

In [140]:
first

,DESC,TAG
0,"Apples,Bananas,Fruits","a,b,f"
1,Bananas,b
2,"Citron,Guava,Apple","c,g,a"
3,"Bananas,Dilithium,Fruits,Hottentot","b,d,f,h"
4,Hottentot,h
5,"Citron,Kiwi","c,k"


In [141]:
def search_func_01(row):
    matches = [item in row["TAG"] for item in checklist]
    if any(matches):
        return "Yes"
    else:
        return "No"    

In [142]:
for item in checklist:
    print (item)

a
b
c
d
e
h
i
j
k
l


In [143]:
first["MATCH"] = first.apply(search_func_01,axis = 1)

In [144]:
first

,DESC,TAG,MATCH
0,"Apples,Bananas,Fruits","a,b,f",Yes
1,Bananas,b,Yes
2,"Citron,Guava,Apple","c,g,a",Yes
3,"Bananas,Dilithium,Fruits,Hottentot","b,d,f,h",Yes
4,Hottentot,h,Yes
5,"Citron,Kiwi","c,k",Yes


In [145]:
def search_func_02(row):
    matches = [item in row["TAG"] for item in checklist]
    if any(matches):
        return matches
    else:
        return "na"

In [146]:
first["MATCH02"] = first.apply(search_func_02,axis = 1)

In [147]:
first

,DESC,TAG,MATCH,MATCH02
0,"Apples,Bananas,Fruits","a,b,f",Yes,"[True, True, False, False, False, False, False..."
1,Bananas,b,Yes,"[False, True, False, False, False, False, Fals..."
2,"Citron,Guava,Apple","c,g,a",Yes,"[True, False, True, False, False, False, False..."
3,"Bananas,Dilithium,Fruits,Hottentot","b,d,f,h",Yes,"[False, True, False, True, False, True, False,..."
4,Hottentot,h,Yes,"[False, False, False, False, False, True, Fals..."
5,"Citron,Kiwi","c,k",Yes,"[False, False, True, False, False, False, Fals..."


In [148]:
vect = CountVectorizer(min_df=1, lowercase=True)

In [149]:
X = vect.fit_transform(first['DESC'])
types_fruit = vect.get_feature_names()

In [150]:
types_fruit

['apple',
 'apples',
 'bananas',
 'citron',
 'dilithium',
 'fruits',
 'guava',
 'hottentot',
 'kiwi']

In [151]:
def search_func_03(row):
    matches = [item in row["TAG"] for item in checklist]
    if any(matches):
        for tag in row["TAG"]:
            print (tag)
            f = [x for x in checklist if x in tag]
            return f
#         contained = [x for x in d if x in paid[j]]
#         for thing in checklist:
#             row["TAG"].str.extract(thing,expand = True)
    else:
        return "na"

In [152]:
# df['dd'] = df.apply(lambda x: x.bb in x.aa, axis=1)
# print (df)
first["MATCH03"] = first.apply(search_func_03, axis = 1)

a
b
c
b
h
c


In [153]:
first


,DESC,TAG,MATCH,MATCH02,MATCH03
0,"Apples,Bananas,Fruits","a,b,f",Yes,"[True, True, False, False, False, False, False...",[a]
1,Bananas,b,Yes,"[False, True, False, False, False, False, Fals...",[b]
2,"Citron,Guava,Apple","c,g,a",Yes,"[True, False, True, False, False, False, False...",[c]
3,"Bananas,Dilithium,Fruits,Hottentot","b,d,f,h",Yes,"[False, True, False, True, False, True, False,...",[b]
4,Hottentot,h,Yes,"[False, False, False, False, False, True, Fals...",[h]
5,"Citron,Kiwi","c,k",Yes,"[False, False, True, False, False, False, Fals...",[c]


In [154]:
'''
>>> import re
>>> list=['a cat','a dog','a yacht','cats']
>>> regex=re.compile(".*(cat).*")
>>> [m.group(0) for l in list for m in [regex.search(l)] if m]
['a cat', 'cats']
>>> [m.group(1) for l in list for m in [regex.search(l)] if m]
['cat', 'cat'
'''

'\nimport re\nlist=[\'a cat\',\'a dog\',\'a yacht\',\'cats\']\nregex=re.compile(".*(cat).*")\n[m.group(0) for l in list for m in [regex.search(l)] if m]\n[\'a cat\', \'cats\']\n[m.group(1) for l in list for m in [regex.search(l)] if m]\n[\'cat\', \'cat\'\n'

In [155]:
def search_func_04(row):
    matches = [item in row["TAG"] for item in checklist]
    if any(matches):
        s1 = set(row["TAG"])
#         print (s1)
        s2 = set (checklist)
#         print (s2)
        return (s1.intersection(s2))
    else:
        return "na"

In [157]:
first["MATCH03"] = first.apply(search_func_04, axis = 1)
first

,DESC,TAG,MATCH,MATCH02,MATCH03
0,"Apples,Bananas,Fruits","a,b,f",Yes,"[True, True, False, False, False, False, False...","{b, a}"
1,Bananas,b,Yes,"[False, True, False, False, False, False, Fals...",{b}
2,"Citron,Guava,Apple","c,g,a",Yes,"[True, False, True, False, False, False, False...","{c, a}"
3,"Bananas,Dilithium,Fruits,Hottentot","b,d,f,h",Yes,"[False, True, False, True, False, True, False,...","{d, b, h}"
4,Hottentot,h,Yes,"[False, False, False, False, False, True, Fals...",{h}
5,"Citron,Kiwi","c,k",Yes,"[False, False, True, False, False, False, Fals...","{c, k}"
